# Move the freight trips data from AZDOT to the Sun Cloud TDM network. 

Define workspace and data.

In [ ]:
import arcpy
arcpy.env.overwriteOutput = True

# authentication to access secured data
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
gis = GIS('pro')
target_sr = arcpy.SpatialReference(2223)
db = arcpy.env.workspace = r".\data\atri_freight.gdb"
arcpy.Describe(db)



Project the data.

In [ ]:
# project in_data to match the target spatial reference
def project(in_data, target_sr, out_name):


    in_sr = arcpy.Describe(in_data).spatialReference

    tr = arcpy.ListTransformations (in_sr, target_sr)
    datum_conversion = ''
    if(len(tr)>0):
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]

    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

Project the data.

In [ ]:
source = 'SunCloudBaseYearNetworkWithoutAttrV2_wATRI202204vol'
project(source, target_sr, 'source_project')

destination = 'SunCloudBaseandFutureVolumesCapacityV3'
project(destination, target_sr, 'destination_project')

Spatial join: share a line segment. 

In [ ]:
joined = arcpy.management.AddSpatialJoin(
    target_features="destination_project",
    join_features="source_project",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping=r'SUNCLOUD_I "SUNCLOUD_I" true true false 16 Text 0 0,First,#,C:\Development\sun_cloud\layers\atri\data\atri_freight.gdb\source_project,SUNCLOUD_I,0,16;count "count" true true false 4 Long 0 0,First,#,C:\Development\sun_cloud\layers\atri\data\atri_freight.gdb\source_project,count,-1,-1',
    match_option="SHARE_A_LINE_SEGMENT_WITH",
    search_radius=None,
    distance_field_name=""
)

In [ ]:
arcpy.ExportFeatures_conversion(joined, 'atri_freight')

Delete where count data is not available.

In [ ]:
selected = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="atri_freight",
    selection_type="NEW_SELECTION",
    where_clause="count IS NULL",
    invert_where_clause=None
)

print(arcpy.GetCount_management(selected)[0])

In [ ]:
arcpy.DeleteFeatures_management(selected)

Standardize field name.

In [ ]:
target_fields = [
    ('count', 'freight_trips', 'Freight Trips (ATRI)'),

]
for current, new, alias in target_fields:
    arcpy.management.AlterField("atri_freight", current, new, alias)

Delete non-key fields.

In [ ]:
fc = 'atri_freight'
keep = ['freight_trips']
f_names = [f.name for f in arcpy.ListFields(fc) if f.required == False and f.name not in keep]
f_names

arcpy.DeleteField_management(fc, f_names)

In [23]:
arcpy.management.DeleteField(fc, f_names)

<Result '.\\data\\atri_freight.gdb\\atri_freight'>